In [12]:
import pandas as pd
import requests

from datetime import datetime
import time

from scrapy import Selector

In [7]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

html = requests.get(url).text
sel = Selector(text = html)

nasdaq100_list = []

for n in range(2,104):
    ticker = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[2]'.format(n)).extract()[0].strip('</td>')
    nasdaq100_list.append(ticker)

In [8]:
%pprint

nasdaq100_list

Pretty printing has been turned OFF


['ATVI', 'ADBE', 'AMD', 'ALGN', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'TEAM', 'ADSK', 'ADP', 'BIDU', 'BIIB', 'BKNG', 'AVGO', 'CDNS', 'CDW', 'CERN', 'CHTR', 'CHKP', 'CTAS', 'CSCO', 'CTSH', 'CMCSA', 'CPRT', 'COST', 'CRWD', 'CSX', 'DXCM', 'DOCU', 'DLTR', 'EBAY', 'EA', 'EXC', 'FAST', 'FISV', 'FOXA', 'FOX', 'GILD', 'HON', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG', 'JD', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LULU', 'MAR', 'MRVL', 'MTCH', 'MELI', 'FB', 'MCHP', 'MU', 'MSFT', 'MRNA', 'MDLZ', 'MNST', 'NTES', 'NFLX', 'NVDA', 'NXPI', 'ORLY', 'OKTA', 'PCAR', 'PAYX', 'PYPL', 'PTON', 'PEP', 'PDD', 'QCOM', 'REGN', 'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK', 'SBUX', 'SNPS', 'TMUS', 'TSLA', 'TXN', 'TCOM', 'VRSN', 'VRSK', 'VRTX', 'WBA', 'WDAY', 'XEL', 'XLNX', 'ZM']

In [13]:
def unix_to_date(unix_timestamp):
    ts = int(unix_timestamp)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [9]:
with open('/Users/ilya/Desktop/keys/yahoo_finance.txt','r') as file:
    key = file.readlines()

KEY = key[0]

In [14]:
def get_weekly_history_df(symbol):
    get_history_url = "https://yh-finance.p.rapidapi.com/stock/v3/get-historical-data"

    querystring = {"symbol":symbol,"region":"US"}

    headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': KEY
    }

    response = requests.request("GET", get_history_url, headers=headers, params=querystring)

    json_response = response.json()

    weekly_history_df = pd.DataFrame(json_response['prices'])[['date','close']]
    weekly_history_df.loc[:,'date'] = weekly_history_df['date'].apply(lambda x: unix_to_date(x))

    weekly_history_df.loc[:,'week'] = [pd.Timestamp(date).week for date in weekly_history_df['date']]
    weekly_history_df.loc[:,'year'] = [pd.Timestamp(date).year for date in weekly_history_df['date']]

    weekly_history_df.loc[:,'close'] = weekly_history_df['close'].round(1)

    return weekly_history_df[['date','year','week','close']]

In [15]:
get_weekly_history_df('GOOG').iloc[0]['close']

2999.1

In [30]:
def list_perfomance(symbol):
    weekly_history_df = get_weekly_history_df(symbol)

    performance_list = [symbol,\
                        round((weekly_history_df.iloc[0]['close'] - weekly_history_df.iloc[-1]['close'])/weekly_history_df.loc[0]['close'],3),\
                        round(weekly_history_df['close'].std(),2),\
                        round(weekly_history_df['close'].var(),2),
                        round(weekly_history_df['close'].mean(),2),
                        round(weekly_history_df['close'].std()/weekly_history_df['close'].mean(),2),
                        round(weekly_history_df.iloc[0]['close'],1)
                        ]

    return performance_list

In [31]:
list_perfomance('GOOG')

['GOOG', 0.419, 402.38, 161911.84, 2383.77, 0.17, 2999.1]

In [32]:
counter = 0

attributes = ['ticker','yoy_growth','std','variance','mean_price','volatility','last_close_price']
comparison_df = pd.DataFrame(columns = attributes)

for ticker_name in nasdaq100_list[64:67]:
    series_row = pd.Series(list_perfomance(ticker_name), index = attributes)
    comparison_df = comparison_df.append(series_row, ignore_index=True)
    print(ticker_name, end = ' ')
    time.sleep(1)

MU MSFT MRNA 

In [33]:
comparison_df.sort_values(by = 'yoy_growth', ascending=False).head(20)

,ticker,yoy_growth,std,variance,mean_price,volatility,last_close_price
2,MRNA,0.630,104.50,10921.20,231.41,0.45,263.8
1,MSFT,0.387,34.87,1215.59,263.00,0.13,343.1
0,MU,0.260,7.47,55.84,78.64,0.10,83.0
